# Semantic Search

1. Read data from a document  
2. Learn how LLMs represent data using embeddings  
3. Understand how embeddings are stored for future retrieval using vector databases

## Setup

In [3]:
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

assert os.environ["LANGSMITH_TRACING"] is not None
assert os.environ["LANGSMITH_API_KEY"] is not None
assert os.environ["LANGSMITH_PROJECT"] is not None
assert os.environ["OPENAI_API_KEY"] is not None

In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

## 1. Documents

- LangChain implements a [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) abstraction, which is intended to represent a unit of text and associated metadata. 
- It has three attributes:
    - **page_content**: a string representing the content;
    - **metadata**: a dict containing arbitrary metadata;
    - **id**: (optional) a string identifier for the document.
- **IMPORTANT NOTE**: A Document object often represents a chunk of a larger document.

### 1.1 Metadata

- The metadata attribute can capture information about the source of the document (e.g. the original PDF document the text is from, page number and so on), its relationship to other documents, and other information. 

In [5]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

## 2. Loading Documentsfrom langchain_community.document_loaders import PyPDFLoader

We will load a PDF Document. 

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/2-Semantic-Search/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


- Note that the number of documents is ~107 i.e. A digital docuemnt consists of a number of LangChain Document Objects.
- PyPDFLoader loads one Document object per PDF page. For each, we can easily access
- **107 document object therefore means 107 pages in the original document**

In [7]:
print(f"{docs[0].page_content[:200]}\n")

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F



PyPDFLoader automatically adds document source and page number in the metadata

In [8]:
import json
print(json.dumps(docs[0].metadata,indent=2))

{
  "producer": "EDGRpdf Service w/ EO.Pdf 22.0.40.0",
  "creator": "EDGAR Filing HTML Converter",
  "creationdate": "2023-07-20T16:22:00-04:00",
  "title": "0000320187-23-000039",
  "author": "EDGAR Online, a division of Donnelley Financial Solutions",
  "subject": "Form 10-K filed on 2023-07-20 for the period ending 2023-05-31",
  "keywords": "0000320187-23-000039; ; 10-K",
  "moddate": "2023-07-20T16:22:08-04:00",
  "source": "./example_data/nke-10k-2023.pdf",
  "total_pages": 107,
  "page": 0,
  "page_label": "1"
}


### 1.2 Document Splitting

- As mentioned, PyPDFLoader maps each page of a PDF to a `Document` object.
- It is preferable to split `Document` objects into smaller chunks, since LLMs process each Document as a single unit of context.
- For example, if a `Document` object contains an entire page on the Health Insurance Policy, the LLM will consider the full page when answering a question about a specific detail. This can **reduce precision** and **increase the likelihood of hallucinations**.


- For general-purpose use cases, `RecursiveCharacterTextSplitter` is the recommended splitter.
- It accepts two key arguments:
    - **chunk_size**: Maximum number of characters per chunk.
    - **chunk_overlap**: Number of characters repeated from the end of the previous chunk at the start of the next. This helps preserve context across chunks.

- The 200-character overlap occurs **between** chunks — at the end of one and the beginning of the next. For example, if your text is 3000 characters, the chunks will be:
    - Chunk 1: Characters 0–999  
    - Chunk 2: Characters 800–1799  
    - Chunk 3: Characters 1600–2599  
    - Chunk 4: Characters 2400–2999

- The splitter uses natural boundaries, such as line breaks or punctuation, to avoid cutting text in unnatural places.

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

In [16]:
[i.__dict__ for i in all_splits[0:3]]

[{'id': None,
  'metadata': {'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
   'creator': 'EDGAR Filing HTML Converter',
   'creationdate': '2023-07-20T16:22:00-04:00',
   'title': '0000320187-23-000039',
   'author': 'EDGAR Online, a division of Donnelley Financial Solutions',
   'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31',
   'keywords': '0000320187-23-000039; ; 10-K',
   'moddate': '2023-07-20T16:22:08-04:00',
   'source': './example_data/nke-10k-2023.pdf',
   'total_pages': 107,
   'page': 0,
   'page_label': '1',
   'start_index': 0},
  'page_content': "Table of Contents\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE FISCAL YEAR ENDED MAY 31, 2023\nOR\n☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE TRANSITION PERIOD FROM                         T

## 2. Embeddings

- Imagine being able to capture the essence of any text - a tweet, document, or book - in a single, compact representation. This is the power of embedding models
- Embedding models transform human language into a format that machines can understand and compare with speed and accuracy.
- These models take text as input and produce a fixed-length array of numbers, a numerical fingerprint of the text's semantic meaning.
- Embeddings allow search system to find relevant documents not just based on keyword matches, but on semantic understanding.

![](./docs/embeddings_concept.png)

1. Embed text as a vector: Embeddings transform text into a numerical vector representation.
1. Measure similarity: Embedding vectors can be compared using simple mathematical operations.

- Each embedding is essentially a set of coordinates, often in a high-dimensional space.
- In this space, the position of each point (embedding) reflects the meaning of its corresponding text.
- Just as similar words might be close to each other in a thesaurus, similar concepts end up close to each other in this embedding space.
- This allows for intuitive comparisons between different pieces of text. By reducing text to these numerical representations, we can use simple mathematical operations to quickly measure how alike two pieces of text are, regardless of their original length or structure

LangChain provides a universal interface for working with embeddings, providing standard methods for common operations. This common interface simplifies interaction with various embedding providers through two central methods:

    embed_documents: For embedding multiple texts (documents)
    embed_query: For embedding a single text (query)

This distinction is important, as some providers employ different embedding strategies for documents (which are to be searched) versus queries (the search input itself). 

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2) # An embedding is a fixed-length array
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.009286928921937943, -0.015886522829532623, 0.0003275189083069563, 0.00636835815384984, 0.020573733374476433, -0.03912259265780449, -0.007468290627002716, 0.040972478687763214, -0.007899514399468899, 0.05984631925821304]


## Vector Stores

- A vector store is a specialised database for storing and querying embeddings. It supports standard operations like create, read, update, and delete (CRUD).
- For retrieval, it allows searching for semantically similar text by comparing embedding vectors using similarity metrics such as cosine similarity.

- VectorStore includes methods for querying:
    - Synchronously and asynchronously;
    - By string query and by vector;
    - With and without returning similarity scores;
    - By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).

- The methods will generally include a list of Document objects in their outputs.

In [37]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
ids = vector_store.add_documents(documents=all_splits)
ids[:3]

['20cd1580-0d1d-44dd-8522-0fefe27ae2da',
 'd61b82b9-63c1-4386-bade-83b4cbae20f8',
 'b2ea1b41-5c92-4569-90df-5fc0fd543414']

- As mentioned earlier, Embeddings typically represent text as a "dense" vector such that texts with similar meanings are geometrically close.
- This lets us retrieve relevant information just by passing in a question, without knowledge of any specific key-terms used in the document.

In [30]:
# Return documents based on similarity to an embedded query:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

## Returns Document object whose semantic meaning closest to the query
print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 

In [31]:
# Note that providers implement different scores; the score here is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6882491406406019

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

In [33]:
# Return documents based on similarity to an embedded query:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

## Retrievers

- LangChain `VectorStore` objects are not `Runnable` instances so they do not support standard `Runnable` methods like `.invoke()` or batch operations.
- `Retriever` objects are `Runnable`s and implement these standard methods.
- `Retriever`s can be built on top of vector stores to provide a unified interface.
- `Retriever`s can also interface with other data sources like external APIs, not just vector stores.
- This approach allows flexible integration of custom retrieval logic into LangChain pipelines.

- You don’t need to subclass Retriever to create one.
- You can manually create a retriever by defining a document retrieval method (e.g., similarity_search) and wrapping it in a runnable.


In [35]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='8b515aef-8099-46bc-a276-f29a119ecd88', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:


In [36]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='8b515aef-8099-46bc-a276-f29a119ecd88', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': './example_data/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and

VectorStoreRetriever supports search types of "similarity" (default), "mmr" (maximum marginal relevance, described above), and "similarity_score_threshold". We can use the latter to threshold documents output by the retriever by similarity score.